# ETL

## Extract

In [1]:
import pandas as pd
import numpy as np

script to extract data from csv, json and parquet files

In [2]:
#import os
#import json
#import pyarrow as pa
#from pathlib import Path
# this script lets you loop through the data folder to read each csv and pass in information into a dataframe named data

#folder = "data/"
#dataframes = []
#for file in os.listdir(folder):
#    path = os.path.join(folder,file)
#    
#    if file.endswith('.csv'):
#        df = pd.read_csv(path)
#    elif file.endswith('parquet'):
#        df = pd.read_parquet(path)
#    else:
#        df = pd.read_json(path, lines=True)
#    
#    print(f"loaded {file} of shape : {df.shape}")
#    dataframes.append(df)
#    
#stores all dataframes into one dataframe no repeat index,
#data = pd.concat(dataframes, ignore_index=True) 


saving data into completeData.csv to prevent repeat extraction

In [3]:
# saving the df as a file so we dont need to repeatedly perform the extraction step
#data.to_csv('completeData.csv', index=False) 
data = pd.read_csv('completeData.csv')

## Transform

In [4]:
#displaying the shape of our data as well as the first rows
data.shape , data.head()

((54593, 79),
     Destination Port   Flow Duration   Total Fwd Packets  \
 0              36102              23                   2   
 1                 53           30805                   2   
 2                443         5535509                   8   
 3                 53           49235                   4   
 4                 53             181                   2   
 
     Total Backward Packets  Total Length of Fwd Packets  \
 0                        1                           31   
 1                        2                          108   
 2                        8                          372   
 3                        2                          152   
 4                        2                           70   
 
     Total Length of Bwd Packets   Fwd Packet Length Max  \
 0                             6                      31   
 1                           230                      54   
 2                          3876                     191   
 3              

In [5]:
# display datas type and non empty values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54593 entries, 0 to 54592
Data columns (total 79 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0    Destination Port             54593 non-null  int64  
 1    Flow Duration                54593 non-null  int64  
 2    Total Fwd Packets            54593 non-null  int64  
 3    Total Backward Packets       54593 non-null  int64  
 4   Total Length of Fwd Packets   54593 non-null  int64  
 5    Total Length of Bwd Packets  54593 non-null  int64  
 6    Fwd Packet Length Max        54593 non-null  int64  
 7    Fwd Packet Length Min        54593 non-null  int64  
 8    Fwd Packet Length Mean       54593 non-null  float64
 9    Fwd Packet Length Std        54593 non-null  float64
 10  Bwd Packet Length Max         54593 non-null  int64  
 11   Bwd Packet Length Min        54593 non-null  int64  
 12   Bwd Packet Length Mean       54593 non-null  float64
 13   

In [6]:
# display our datas empty values
data.isna().sum()

 Destination Port              0
 Flow Duration                 0
 Total Fwd Packets             0
 Total Backward Packets        0
Total Length of Fwd Packets    0
                              ..
Idle Mean                      0
 Idle Std                      0
 Idle Max                      0
 Idle Min                      0
 Label                         0
Length: 79, dtype: int64

In [7]:
#import seaborn as sns
#import matplotlib as plt
#sns.pairplot(data, kind="scatter")

#using sns pairplot we can visualize data corelation and decide what to 

In [8]:

# we also want to make clear what our labels are so we will assign attack as the new name
data = data.rename(columns={" Label": "attack"})
data.columns[-1]

'attack'

the dataset includes different kinds of attacks

- Benign
- Dos GoldenEye
- Dos Hulk
- Dos Slowhttptest
- Dos slowloris
- Heartbleed

we know Heartbleed is not a denial of service attack and can be removed from our attack data

In [9]:
# viewing the different type of attacks
data['attack'].unique()

array(['BENIGN', 'DoS Hulk', 'DoS GoldenEye'], dtype=object)

In [10]:
# we apply 1 for labels which contain dos as part of the label
data['attack'] = data['attack'].apply(lambda x: 1 if "dos" in str(x).lower() else 0 )
data['attack'].tail()

54588    1
54589    1
54590    1
54591    1
54592    1
Name: attack, dtype: int64

In [11]:
# showing the values in our data which have missing values
data[data.isna().any(axis=1)]

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,attack
5245,80,0,2,0,0,0,0,0,0.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,1
5365,80,0,2,0,0,0,0,0,0.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,1
5744,80,0,2,0,0,0,0,0,0.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,1
5797,80,0,2,0,0,0,0,0,0.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,1
5867,80,0,2,0,0,0,0,0,0.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32829,80,0,2,0,0,0,0,0,0.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,1
33339,80,0,2,0,0,0,0,0,0.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,1
33415,80,0,2,0,0,0,0,0,0.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,1
33692,80,0,2,0,0,0,0,0,0.0,0.0,...,32,0.0,0.0,0,0,0.0,0.0,0,0,1


In [12]:
data.max()

 Destination Port                  63849.0
 Flow Duration                 119998998.0
 Total Fwd Packets                  5117.0
 Total Backward Packets             8350.0
Total Length of Fwd Packets        88632.0
                                  ...     
Idle Mean                      120000000.0
 Idle Std                       60700000.0
 Idle Max                      120000000.0
 Idle Min                      120000000.0
attack                                 1.0
Length: 79, dtype: float64

In [13]:
# 
column_variances = data.var()

filtered_data = data.loc[:, column_variances != 0]
filtered_data = filtered_data.dropna(axis=1)

for x ,col in enumerate(filtered_data.columns):
    print(x," ",col," : var ",filtered_data[col].var()," : mean " ,filtered_data[col].mean())


0    Destination Port  : var  41825497.60296314  : mean  933.5089663509974
1    Flow Duration  : var  1830461805440085.5  : mean  40233831.92130859
2    Total Fwd Packets  : var  793.9157848016172  : mean  5.796695547048157
3    Total Backward Packets  : var  1979.1633218286488  : mean  4.474254941109666
4   Total Length of Fwd Packets  : var  1062261.7598755558  : mean  367.60938215522134
5    Total Length of Bwd Packets  : var  5866305219.182368  : mean  7592.294524939095
6    Fwd Packet Length Max  : var  92473.73401956048  : mean  265.11708460791675
7    Fwd Packet Length Min  : var  226.39299789179387  : mean  2.1113146374077263
8    Fwd Packet Length Mean  : var  6108.210461470073  : mean  52.63134568792843
9    Fwd Packet Length Std  : var  15088.063840332297  : mean  107.26616948427852
10   Bwd Packet Length Max  : var  10909845.074229602  : mean  3761.430513069441
11    Bwd Packet Length Min  : var  736.5414894958412  : mean  4.855494294140274
12    Bwd Packet Length Mean  : v

In [14]:
# displaying some information about the data which has been removed to check if there is any information we should reconsider
removed_data = data.loc[:, column_variances == 0 ]
removed_data

,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,CWE Flag Count,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
54588,0,0,0,0,0,0,0,0,0,0
54589,0,0,0,0,0,0,0,0,0,0
54590,0,0,0,0,0,0,0,0,0,0
54591,0,0,0,0,0,0,0,0,0,0


In [15]:
# viewing if our data is standardized (mean = 0, std =1 )
filtered_data.describe()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,attack
count,54593.000000,5.459300e+04,54593.000000,54593.000000,54593.000000,5.459300e+04,54593.000000,54593.000000,54593.000000,54593.000000,...,54593.000000,5.459300e+04,5.459300e+04,5.459300e+04,5.459300e+04,5.459300e+04,5.459300e+04,5.459300e+04,5.459300e+04,54593.000000
mean,933.508966,4.023383e+07,5.796696,4.474255,367.609382,7.592295e+03,265.117085,2.111315,52.631346,107.266169,...,29.053688,8.601111e+04,1.374292e+04,1.006066e+05,7.689053e+04,3.735111e+07,4.621719e+05,3.768663e+07,3.701618e+07,0.908322
std,6467.263533,4.278390e+07,28.176511,44.487788,1030.660836,7.659181e+04,304.094942,15.046362,78.155041,122.833480,...,5.213573,5.799821e+05,2.330139e+05,6.778768e+05,5.486710e+05,4.304798e+07,5.060585e+06,4.333113e+07,4.306638e+07,0.288574
min,0.000000,0.000000e+00,1.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,80.000000,1.454820e+05,2.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,32.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000
50%,80.000000,1.180376e+07,6.000000,5.000000,340.000000,1.159500e+04,329.000000,0.000000,47.500000,125.933018,...,32.000000,6.000000e+00,0.000000e+00,6.000000e+00,6.000000e+00,6.792433e+06,0.000000e+00,6.792784e+06,6.745733e+06,1.000000
75%,80.000000,8.581424e+07,8.000000,6.000000,395.000000,1.159500e+04,374.000000,0.000000,66.400000,156.553505,...,32.000000,1.038000e+03,0.000000e+00,1.038000e+03,1.031000e+03,8.520000e+07,0.000000e+00,8.530000e+07,8.520000e+07,1.000000
max,63849.000000,1.199990e+08,5117.000000,8350.000000,88632.000000,1.450000e+07,11595.000000,1983.000000,2319.000000,5185.441640,...,48.000000,1.370000e+07,6.905038e+06,1.920000e+07,1.370000e+07,1.200000e+08,6.070000e+07,1.200000e+08,1.200000e+08,1.000000


In [22]:
# x will contain the information outside of the labels for each sample
x = filtered_data.loc[:,filtered_data.columns != 'attack']
x

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,36102,23,2,1,31,6,31,0,15.500000,21.920310,...,0,32,0.0,0.0,0,0,0.0,0.0,0,0
1,53,30805,2,2,108,230,54,54,54.000000,0.000000,...,1,20,0.0,0.0,0,0,0.0,0.0,0,0
2,443,5535509,8,8,372,3876,191,0,46.500000,71.876879,...,7,20,346981.0,0.0,346981,346981,5188524.0,0.0,5188524,5188524
3,53,49235,4,2,152,202,38,38,38.000000,0.000000,...,3,32,0.0,0.0,0,0,0.0,0.0,0,0
4,53,181,2,2,70,218,35,35,35.000000,0.000000,...,1,20,0.0,0.0,0,0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54588,80,11512204,8,5,326,11632,326,0,40.750000,115.258405,...,1,32,892.0,0.0,892,892,6507197.0,0.0,6507197,6507197
54589,80,11513325,5,5,471,3525,471,0,94.200000,210.637604,...,1,32,918.0,0.0,918,918,6508582.0,0.0,6508582,6508582
54590,80,11509201,7,6,314,11632,314,0,44.857143,118.680845,...,1,32,899.0,0.0,899,899,6503248.0,0.0,6503248,6503248
54591,80,11509095,8,5,369,11632,369,0,46.125000,130.461201,...,1,32,914.0,0.0,914,914,6504954.0,0.0,6504954,6504954


In [17]:
col_std_data = filtered_data.loc[:,filtered_data.columns.str.contains("Std")]

col_std_data

,Fwd Packet Length Std,Bwd Packet Length Std,Flow IAT Std,Fwd IAT Std,Bwd IAT Std,Packet Length Std,Active Std,Idle Std
0,21.920310,0.000000,1.202082e+01,0.000000e+00,0.000000e+00,16.350331,0.0,0.0
1,0.000000,0.000000,1.778008e+04,0.000000e+00,0.000000e+00,33.411076,0.0,0.0
2,71.876879,674.522053,1.334058e+06,1.940029e+06,5.592656e+04,503.409858,0.0,0.0
3,0.000000,0.000000,1.370579e+04,1.623855e+04,0.000000e+00,30.740852,0.0,0.0
4,0.000000,0.000000,9.843949e+01,0.000000e+00,0.000000e+00,40.531469,0.0,0.0
...,...,...,...,...,...,...,...,...
54588,115.258405,4437.056321,2.262616e+06,2.460477e+06,3.378874e+06,2713.257986,0.0,0.0
54589,210.637604,990.644740,2.565153e+06,3.255639e+06,3.379306e+06,719.780257,0.0,0.0
54590,118.680845,2544.918912,2.261842e+06,2.655701e+06,3.195689e+06,1857.187221,0.0,0.0
54591,130.461201,4437.056321,2.262122e+06,2.458891e+06,3.377879e+06,2712.638403,0.0,0.0


In [18]:
np.unique(col_std_data,return_counts=True)

(array([0.00000000e+00, 5.00000000e-01, 5.77350269e-01, ...,
        8.27000000e+07, 8.29000000e+07, 8.33000000e+07], shape=(76133,)),
 array([208926,     11,     10, ...,      2,      1,      1],
       shape=(76133,)))

In [ ]:
for i in filtered_data[' Active Std']:
    print(i)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
94295.17009
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1066385.039
0.0
0.0
0.0
6486086.013
0.0
1218384.589
0.0
0.0
0.0
0.0
0.0
0.0
32620.731
0.0
0.0
0.0
0.0
0.0
4108719.251
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
33410.57621
0.0
0.0
37365.82665
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
20679.78026
0.0
0.0
62863.91417
0.0
0.0
899124.456
0.0
0.0
0.0
0.0
711489.9835
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
11964.82889
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
161367.5383
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
119637.8508
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0

In [20]:
# y will hold only the label values which are stored as true false or 0,1 where the label is considered an attack
y = filtered_data['attack'].copy()
y

0        0
1        0
2        0
3        0
4        0
        ..
54588    1
54589    1
54590    1
54591    1
54592    1
Name: attack, Length: 54593, dtype: int64

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=.3,random_state=42,stratify=y)

In [25]:
from sklearn.preprocessing import StandardScaler
#creating a standard scaler to standardize the filtered data
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [26]:
# displaying if x train is standardized (mean = 0, std = 1)
pd.DataFrame(X_train).describe()

,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,65
count,3.821500e+04,3.821500e+04,3.821500e+04,3.821500e+04,3.821500e+04,3.821500e+04,3.821500e+04,3.821500e+04,3.821500e+04,3.821500e+04,...,3.821500e+04,3.821500e+04,3.821500e+04,38215.000000,3.821500e+04,38215.000000,3.821500e+04,3.821500e+04,3.821500e+04,3.821500e+04
mean,1.041224e-17,3.421166e-17,-1.115598e-18,-7.437318e-19,1.636210e-17,1.859329e-18,7.511691e-17,2.826181e-17,-9.296647e-18,6.321720e-17,...,-3.718659e-19,-2.115917e-16,-4.090525e-18,0.000000,2.603061e-18,0.000000,-1.740332e-16,-9.296647e-18,-1.264344e-17,5.429242e-17
std,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00,...,1.000013e+00,1.000013e+00,1.000013e+00,1.000013,1.000013e+00,1.000013,1.000013e+00,1.000013e+00,1.000013e+00,1.000013e+00
min,-1.439839e-01,-9.386471e-01,-1.508634e-01,-8.689194e-02,-3.326649e-01,-8.631688e-02,-8.570591e-01,-1.383242e-01,-6.615212e-01,-8.576214e-01,...,-5.212674e-02,-5.578625e+00,-1.478583e-01,-0.057629,-1.479825e-01,-0.140173,-8.655176e-01,-9.258336e-02,-8.675480e-01,-8.571453e-01
25%,-1.315545e-01,-9.352448e-01,-1.195697e-01,-8.689194e-02,-3.326649e-01,-8.631688e-02,-8.570591e-01,-1.383242e-01,-6.615212e-01,-8.576214e-01,...,-5.212674e-02,5.645217e-01,-1.478583e-01,-0.057629,-1.479825e-01,-0.140173,-8.655176e-01,-9.258336e-02,-8.675480e-01,-8.571453e-01
50%,-1.315545e-01,-6.629730e-01,5.605277e-03,8.590002e-03,-2.616215e-02,4.352738e-02,2.058798e-01,-1.383242e-01,-6.418446e-02,1.488015e-01,...,-1.988135e-02,5.645217e-01,-1.478480e-01,-0.057629,-1.479737e-01,-0.140162,-7.078379e-01,-9.258336e-02,-7.109448e-01,-7.006311e-01
75%,-1.315545e-01,1.066774e+00,6.819276e-02,2.768639e-02,2.522213e-02,4.352738e-02,3.544974e-01,-1.383242e-01,1.746457e-01,3.964186e-01,...,1.236404e-02,5.645217e-01,-1.460629e-01,-0.057629,-1.464407e-01,-0.138294,1.114439e+00,-9.258336e-02,1.101044e+00,1.122082e+00
max,9.776129e+00,1.865817e+00,1.599479e+02,1.593680e+02,7.956719e+01,1.622890e+02,3.660427e+01,1.304052e+02,2.842455e+01,4.056101e+01,...,1.620777e+02,3.636095e+00,2.341175e+01,30.733965,2.820690e+01,24.669839,1.923153e+00,1.163873e+01,1.901866e+00,1.930498e+00


In [28]:
# displayed if x test is standardized (mean = 0, std = 1)
pd.DataFrame(X_test).describe()

,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,65
count,16378.000000,16378.000000,16378.000000,16378.000000,16378.000000,16378.000000,16378.000000,16378.000000,16378.000000,16378.000000,...,16378.000000,16378.000000,16378.000000,16378.000000,16378.000000,16378.000000,16378.000000,16378.000000,16378.000000,16378.000000
mean,0.003513,0.005490,-0.002523,-0.004833,-0.004243,-0.004320,-0.001714,0.002222,-0.004641,-0.002782,...,-0.005034,-0.003639,0.000177,0.012183,0.001983,-0.003095,0.008273,-0.010869,0.007337,0.009180
std,1.015934,0.999626,0.508169,0.269137,0.737683,0.344635,0.940312,0.968044,0.932566,0.935617,...,0.228496,1.002882,0.991250,1.124950,1.003660,0.978553,1.001272,0.924744,1.000033,1.001462
min,-0.143984,-0.938647,-0.150863,-0.086892,-0.332665,-0.086317,-0.857059,-0.138324,-0.661521,-0.857621,...,-0.052127,-5.578625,-0.147858,-0.057629,-0.147983,-0.140173,-0.865518,-0.092583,-0.867548,-0.857145
25%,-0.131554,-0.935268,-0.119570,-0.086892,-0.332665,-0.086317,-0.857059,-0.138324,-0.661521,-0.857621,...,-0.052127,0.564522,-0.147858,-0.057629,-0.147983,-0.140173,-0.865518,-0.092583,-0.867548,-0.857145
50%,-0.131554,-0.662517,0.005605,0.008590,-0.027064,0.043527,0.205880,-0.138324,-0.067320,0.146142,...,-0.019881,0.564522,-0.147850,-0.057629,-0.147975,-0.140164,-0.707278,-0.092583,-0.710401,-0.699947
75%,-0.131554,1.067663,0.068193,0.027686,0.021616,0.043527,0.351267,-0.138324,0.164194,0.384432,...,-0.019881,0.564522,-0.146096,-0.057629,-0.146469,-0.138324,1.116763,-0.092583,1.103352,1.124405
max,9.720352,1.863153,59.182069,28.805944,41.438250,39.989420,27.212283,95.185464,23.576868,27.146669,...,17.457120,2.868202,18.768613,29.626960,22.447313,19.780274,1.923153,11.638733,1.901866,1.930498


In [29]:
(pd.DataFrame(filtered_data).corr().style.background_gradient(cmap='PRGn',vmin=-1,vmax=1))

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,attack
Destination Port,1.000000,-0.106075,-0.016099,-0.009224,0.079878,-0.012409,0.082732,0.054024,0.150717,0.088364,-0.148897,-0.011658,-0.159192,-0.144758,-0.056265,-0.081868,-0.099711,-0.033398,-0.100177,-0.067898,-0.081353,-0.099038,-0.036802,-0.036391,-0.036168,-0.030047,-0.030318,-0.006205,0.348582,-0.025382,-0.015636,0.038269,0.199597,0.017982,-0.132733,-0.136925,-0.136699,-0.106595,-0.037573,0.348582,-0.000325,-0.088430,0.118716,0.466065,-0.000325,0.055675,-0.134328,0.150717,-0.159192,-0.025382,-0.016099,0.079878,-0.009224,-0.012419,-0.068600,0.169834,-0.003043,-0.080111,-0.017669,-0.005808,-0.017078,-0.017149,-0.098015,-0.008421,-0.098057,-0.097261,-0.415411
Flow Duration,-0.106075,1.000000,0.055185,0.046544,0.097492,0.049356,0.227464,-0.122145,0.147888,0.228143,0.346797,-0.150303,0.390676,0.277271,0.412612,0.852314,0.982654,0.178002,0.998802,0.577888,0.863418,0.982570,0.248927,0.402571,0.321011,0.397348,0.408794,0.034596,-0.059245,0.068126,0.067341,-0.277942,-0.054769,-0.160798,0.352377,0.434514,0.351969,0.231944,0.372438,-0.059245,-0.003625,-0.455227,0.239374,-0.106322,-0.003625,0.115168,0.428474,0.147888,0.390676,0.068126,0.055185,0.097492,0.046544,0.049391,-0.460094,-0.022352,0.034256,-0.389107,-0.042777,0.033156,-0.018276,-0.053812,0.975631,0.154886,0.982248,0.962132,0.207955
Total Fwd Packets,-0.016099,0.055185,1.000000,0.980670,0.555818,0.987945,0.072892,-0.015280,0.028392,0.046753,0.082013,-0.014367,0.082085,0.061432,-0.031935,0.020339,0.024301,-0.034326,0.052085,-0.028051,0.030225,0.023929,-0.037750,0.055859,0.011733,0.010474,0.014915,-0.002987,0.061630,0.984717,0.979702,-0.039034,-0.008717,-0.022535,0.082371,0.081779,0.067527,0.046615,-0.006655,0.061630,0.001246,0.051667,-0.033772,-0.018093,0.001246,-0.003861,0.077993,0.028392,0.082085,0.984717,1.000000,0.555818,0.980670,0.987971,0.043074,0.004668,0.971571,-0.026908,0.050484,0.050311,0.066866,0.038752,0.020061,0.010626,0.020899,0.019037,-0.028182
Total Backward Packets,-0.009224,0.046544,0.980670,1.000000,0.546598,0.996089,0.055454,-0.007754,0.021258,0.036031,0.055015,0.000206,0.054761,0.034022,-0.022216,0.019773,0.021117,-0.025619,0.044864,-0.015327,0.029327,0.020823,-0.026506,0.061758,0.017899,0.021748,0.026490,-0.002330,0.066748,0.939258,0.997972,-0.029653,-0.003924,-0.011835,0.055288,0.066787,0.044654,0.026579,0.013755,0.066748,0.000532,0.029086,-0.031748,-0.009222,0.000532,0.036244,0.064191,0.021258,0.054761,0.939258,0.980670,0.546598,1.000000,0.996093,0.022662,0.004719,0.990359,-0.034887,0.018360,0.023561,0.028824,0.013740,0.018892,0.004368,0.019122,0.018452,-0.033087
Total Length of Fwd Packets,0.079878,0.097492,0.555818,0.546598,1.000000,0.525850,0.546642,0.059051,0.595132,0.508149,0.139913,-0.037739,0.139129,0.121841,-0.071308,0.064733,0.045773,-0.090844,0.089066,-0.048804,0.065474,0.044917,-0.072489,0.156243,0.102610,0.091131,0.091812,0.003996,0.037776,0.549599,0.5

In [30]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay

In [31]:
np.unique(y_test,return_counts=True)

(array([0, 1]), array([ 1502, 14876]))

In [32]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train,y_train)

predictions = model.predict(X_test)

accuracy = accuracy_score(y_test,predictions)

print("model accuracy : ", accuracy)

model accuracy :  0.9971302967395287


In [33]:
from xgboost import XGBClassifier

xg_model = XGBClassifier(n_estimators=150).fit(X_train, y_train, verbose = True)
xg_pred = xg_model.predict(X_test)

xg_model

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [34]:
xg_pred

array([1, 1, 1, ..., 1, 1, 1], shape=(16378,))

In [35]:
# compute the measures
acc_xgboost1 = accuracy_score(y_test, xg_pred)
f1score_xgboost1 = f1_score(y_test, xg_pred)
precision_xgboost1 = precision_score(y_test, xg_pred)
recall_xgboost1 = recall_score(y_test, xg_pred)

# display the measures

print('-'*25)
print('XGBoost Model: 66 features')
print('-'*25)
print('Accuracy: {:.3f}'.format(acc_xgboost1))
print('F1-score: {:.3f}'.format(f1score_xgboost1))
print('Precision: {:.3f}'.format(precision_xgboost1))
print('Recall: {:.3f}'.format(recall_xgboost1))

-------------------------
XGBoost Model: 66 features
-------------------------
Accuracy: 1.000
F1-score: 1.000
Precision: 1.000
Recall: 1.000


In [40]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, xg_pred)

print("Confusion Matrix:")
print("-"*18)
print(conf_matrix)
print("-"*18)
print("tn,fp")
print("fn,tp")

Confusion Matrix:
------------------
[[ 1499     3]
 [    0 14876]]
------------------
tn,fp
fn,tp
